In [4]:
%load_ext autoreload
%autoreload 2
import torch
import json
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data
from ffm_graph import *
from data import MINDDataset
from data_utils import *
from transformers import BertConfig
from gnn import *
from metrics import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
train_data_path = 'data/MINDsmall_train/'
valid_data_path = 'data/MINDsmall_dev/'

mind_train_dataset = MINDDataset(train_data_path + 'behaviors.pkl')
train_dataloader = DataLoader(mind_train_dataset, batch_size=64, collate_fn=collate_fn, num_workers=4)

In [6]:
mind_valid_dataset = MINDDataset(valid_data_path + 'behaviors.pkl')
valid_dataloader = DataLoader(mind_valid_dataset, batch_size=32, collate_fn=valid_collate_fn, num_workers=4)

In [7]:
news_encoder_config = BertConfig.from_json_file('news_encoder.json')
bert_features_path = valid_data_path + 'bert_features.pt'
news_encoder = NewsEncoder(news_encoder_config, bert_features_path)

user_encoder_config = BertConfig.from_json_file('user_encoder.json')
user_encoder = UserEncoder(user_encoder_config)

gnn = create_sage(nfeat=256, nhid=256, dropout=0.1, nlayer=3)

In [8]:
device = torch.device('cuda')
model = Fastformer_Graph_Lite(news_encoder, gnn).to(device)

In [9]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {total_params}')

Total number of parameters: 1365057


In [10]:
def train_model(model, train_loader, global_graph_data, device, epochs=5):
	optimizer = optim.AdamW(model.parameters(), lr=1e-4)
	criterion = torch.nn.CrossEntropyLoss()
	
	model.to(device)

	for epoch in range(epochs):
		model.train()
		total_loss = 0
		pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
		for batch in pbar:
			batch = {k: v.to(device) for k, v in batch.items()}

			sub_loader = NeighborLoader(
				global_graph_data,
				num_neighbors=[3, 3, 3],
				input_nodes=batch['seed_nodes'].cpu(),
				batch_size=len(batch['seed_nodes']),
				shuffle=False
			)
			sub_graph = next(iter(sub_loader)).to(device)

			optimizer.zero_grad()

			scores = model(batch, sub_graph, device)
			loss = criterion(scores, batch['label'])

			loss.backward()
			optimizer.step()

			total_loss += loss.item()
			pbar.set_postfix({"loss": f"{loss.item():.4f}"})
		torch.save(model.state_dict(), "ffmg_large.pt")
		print(f'Epoch {epoch} done! AVG loss: {total_loss / len(train_loader):.4f}')

In [11]:
def validate_model(model, valid_loader, global_graph_data, device):
    model.eval()
    model.to(device)
    all_auc, all_mrr = [], []
    
    with torch.no_grad():
        pbar = tqdm(valid_loader)
        for batch in pbar:
            batch_gpu = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
            
            sub_loader = NeighborLoader(
                global_graph_data,
                num_neighbors=[3, 3, 3],
                input_nodes=batch['seed_nodes'],
                batch_size=len(batch['seed_nodes']),
                shuffle=False
            )
            sub_graph = next(iter(sub_loader)).to(device)


            scores = model(batch_gpu, sub_graph, device).cpu().numpy()
            labels = batch['labels'].numpy() 

            for i in range(len(labels)):
                valid_mask = labels[i] != -1
                actual_scores = scores[i][valid_mask]
                actual_labels = labels[i][valid_mask]

                if len(np.unique(actual_labels)) > 1:
                    all_auc.append(roc_auc_score(actual_labels, actual_scores))
                    all_mrr.append(mrr_score(actual_labels, actual_scores))
            pbar.set_postfix_str(f"auc: {np.mean(all_auc):.4f}, mrr: {np.mean(all_mrr):.4f}")
    print(f"AUC: {np.mean(all_auc):.4f}, MRR: {np.mean(all_mrr):.4f}")

In [14]:
model_state_dict = torch.load('ffmg_large.pt')

In [15]:
model.load_state_dict(model_state_dict)

RuntimeError: Error(s) in loading state_dict for Fastformer_Graph_Lite:
	Unexpected key(s) in state_dict: "user_encoder.fastformer.encoders.0.attention.self.query.weight", "user_encoder.fastformer.encoders.0.attention.self.query.bias", "user_encoder.fastformer.encoders.0.attention.self.query_att.weight", "user_encoder.fastformer.encoders.0.attention.self.query_att.bias", "user_encoder.fastformer.encoders.0.attention.self.key.weight", "user_encoder.fastformer.encoders.0.attention.self.key.bias", "user_encoder.fastformer.encoders.0.attention.self.key_att.weight", "user_encoder.fastformer.encoders.0.attention.self.key_att.bias", "user_encoder.fastformer.encoders.0.attention.self.transform.weight", "user_encoder.fastformer.encoders.0.attention.self.transform.bias", "user_encoder.fastformer.encoders.0.attention.output.dense.weight", "user_encoder.fastformer.encoders.0.attention.output.dense.bias", "user_encoder.fastformer.encoders.0.attention.output.LayerNorm.weight", "user_encoder.fastform

In [ ]:
def prune_edges_per_node(edge_index, max_edges_per_node=3):
    print(f"📊 Số cạnh gốc: {edge_index.size(1)}")
    

    edge_np = edge_index.cpu().numpy()
    sources = edge_np[0]
    
    mask = []
    counts = {}
    
    for src in sources:
        count = counts.get(src, 0)
        if count < max_edges_per_node:
            mask.append(True)
            counts[src] = count + 1
        else:
            mask.append(False)
            
    mask = torch.tensor(mask)
    new_edge_index = edge_index[:, mask]
    
    print(f"✅ Số cạnh sau khi lọc ({max_edges_per_node} cạnh/nút): {new_edge_index.size(1)}")
    return new_edge_index

In [ ]:
news_tokens = np.load(valid_data_path + 'news_token.npy')
x = torch.from_numpy(news_tokens).long()

edge_index = torch.load(valid_data_path + 'edge_index.pt').contiguous()
edge_index = prune_edges_per_node(edge_index, 3)
valid_graph_data = Data(x=x, edge_index=edge_index)

In [ ]:
validate_model(model, valid_dataloader, valid_graph_data, device)

  0%|          | 0/2286 [00:00<?, ?it/s]Exception ignored in: <function _xla_gc_callback at 0x000001FC89B8CF70>
Traceback (most recent call last):
  File "c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\jax\_src\lib\__init__.py", line 118, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
